In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg
import scipy.ndimage
import scipy.stats as sp
import math

def readData():
    training = np.loadtxt('digits123-1.csv', delimiter=';')
    test = np.loadtxt('digits123-2.csv', delimiter=';')
    return training, test

# Calculate mean for all features in dataset
# return list of all means
def calcMean(X):
    m = len(X)
    n = len(X[0])
    totalSum = []
    for i in range(n):
        sum = 0
        for j in range(m):
            sum += X[j][i]

        totalSum.append(float(sum / len(X)))

    return totalSum

# Calculate variance for all feautures in dataset
# based on list of means and dataset
def calcVar(X, mean):
    m = len(X)
    n = len(X[0])
    totalSum = np.ones((m,n))
    for i in range(n):
        for j in range(m):
            totalSum[j][i] = (X[j][i] - mean[i])**2
    
    # calc mean of variance per feature
    var = calcMean(totalSum)
    
    return var
    
# test function
def tryOut(mean , var):
    total = []
    for i in range(len(mean)):
        if var[i] != 0:
            total.append(mean[i] / var[i])
        else:
            total.append(mean[i])
    return total
         
# returns the size of each class based on target values
def classSize(Y):
    size1 = 0
    size2 = 0
    size3 = 0
    for i in range(len(Y)):
        if Y[i] == 1.0:
            size1 += 1
        elif Y[i] == 2.0: 
            size2 += 1
        else:
            size3 +=1
            
    return [size1, size2, size3]


# calculate normal distribution
def gaussian(mean, var, v):
    if var != 0:
        gaus = 1.0 / math.sqrt(2 * math.pi * var) * math.exp(-(v - mean)**2/(2*var))
    else:
        if v == mean:
            gaus = 1
        else:
            gaus = 0
            
    return gaus

    
# split dataset for each class
def splitClasses(X, Y):
    size = classSize(Y)
    class1 = X[0: size[0]]
    class2 = X[size[0]:size[0] + size[1]]
    class3 = X[size[0] + size[1]:size[0] + size[1] + size[2]]
    return [class1, class2, class3]
    
# retrieve mean for each class
def classMean(classes):
    mean1 = calcMean(classes[0])
    mean2 = calcMean(classes[1])
    mean3 = calcMean(classes[2])
    return [mean1, mean2, mean3]

# retrieve variance for each class
def classVars(classes, means):
    var1 = calcVar(classes[0], means[0])
    var2 = calcVar(classes[1], means[1])
    var3 = calcVar(classes[2], means[2])
    return [var1, var2, var3]

# apply test set on training mean and variance
def training(means, variances, testSet):
    m = len(testSet)
    n = len(testSet[0])
    for i in range(m):
        for j in range(n):
            # class 1
            result = gaussian(means[0][j], variances[0][j] ,testSet[i][j])
            # class 2
            result2 = gaussian(means[1][j], variances[1][j] ,testSet[i][j])
            
#             print result
            
    print means[0][1], variances[0][1], testSet[0][1]
    
dataSets = readData()
X = dataSets[0]
Y = X[:,-1]
X = np.delete(X, -1, 1)

testSet = dataSets[1]
testSet = np.delete(testSet, -1, 1)
# print testSet

classes = splitClasses(X, Y)
means = classMean(classes)
variances = classVars(classes, means)

training(means, variances, testSet)

0.00980392156863 0.0097078046905 0.0


In [10]:
# calculate normal distribution
def gaussian(mean, var, v):
    if var != 0:
        #mean = 0.00980392156863
        #var = 0.0097078046905
        #v = 0.0
        step0 = math.pow((v-mean), 2)
        print step0
        step1 = step0 / (2 * var)
        print step1
        step2 = math.exp(-step1)
        print step2
        
        step3 = 2* math.pi * var
        print step3
        step4 = math.sqrt(step3)
        print step4
        step5 = 1.0 / step4
        print step5
        
        step6 = step2 * step5
        print step6
       
       
        gaus = 1.0 / math.sqrt(2 * math.pi * var) * math.exp(-(v - mean)**2/(2*var))
        
    print 'PDF: ',sp.norm.pdf(v, loc=mean, scale=math.sqrt(var))
    print 'FUNC:',gaus
    return gaus

result = gaussian(0.00980392156863,0.0097078046905,0.0)
result = gaussian(0,0.4472,0)
print result

9.61168781238e-05
0.00495049504951
0.995061738455
0.0609959357963
0.246973552828
4.04901653862
4.02902143595
PDF:  4.02902143595
FUNC: 4.02902143595
0.0
0.0
1.0
2.80984046937
1.67625787675
0.5965669208
0.5965669208
PDF:  0.5965669208
FUNC: 0.5965669208
0.5965669208
